In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ma23m011 (ma23m011-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [19]:
# For visualization import Matplotlib
import matplotlib.pyplot as plt
# For numerical operations import NumPy
import numpy as np
# For interactive UI import ipywidgets
import ipywidgets as widgets
# For logging import Weights & Biases
import wandb
# To show UI elements import display
from IPython.display import display
# Fashion-MNIST dataset import
from keras.datasets import fashion_mnist

# Initialize Weights & Biases project
wandb.init(project="DA6401_Assignment1_ma23m011", name="image_grid")

# Load Fashion-MNIST dataset
(fmnist_train_images, fmnist_train_labels), _ = fashion_mnist.load_data()

# Class labels name defines for Fashion-MNIST data set
ClassLabelNames = ["T-shirt", "Trouser", "Pullover", "Dress", "Coat","Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]


#  To visualize images define function
def VisualizeImage(step=1, index=0):
    fig, axes = plt.subplots(2, 5, figsize=(10, 6))  # Create 2x5 grid images
    fig.suptitle("Fashion-MNIST Dataset Examples", fontsize=14)  # Title set for all printed images

    img = []  # Define empty images list for weight and bias logging

    for j in range(10):
        all_indices = np.where(fmnist_train_labels == j)[0]  # from class j get all indices
        adjust_index = index * step  # Adjusted index computed

        if adjust_index >= len(all_indices):  # Ensure index is within bounds
            adjust_index = len(all_indices) - 1

        image_index = all_indices[adjust_index]  # The final image index

        axis = axes[j // 5, j % 5]  # Subplot position select
        axis.imshow(fmnist_train_images[image_index], cmap='gray')  # Display all images
        axis.set_title(f"{ClassLabelNames[j]}\nIndex: {index}", fontsize=10, pad=4)

        axis.axis("off")  #Axis ticks remove

        # To log in wandb append image
        img.append(wandb.Image(fmnist_train_images[image_index],
                                  caption=f"{ClassLabelNames[j]} (Index: {index})"))

    plt.tight_layout()  #To avoid overlap adjust layout
    plt.show()  # Show the figure

    # Log image grid to weight and bias
    wandb.log({"fashion_mnist_dataset": img, "Step": step, "Index": index})

# Create sliders for step size selection
StepSlider = widgets.IntSlider(value=1, min=0, max=2, step=1, description="Step")

# Create sliders for image index selection
IndexSlider = widgets.IntSlider(value=0, min=1, max=35, step=1, description="Index")

# Group sliders in a UI box
u_i_element = widgets.VBox([StepSlider, IndexSlider])

# Link sliders to the visualization function interactively
output_visualize = widgets.interactive_output(VisualizeImage, {'step': StepSlider, 'index': IndexSlider})

# Display UI elements and output
display(u_i_element, output_visualize)


Output()